### 1.Data Understanding

In [16]:
#import libraries
import pandas as pd

In [17]:
# load data
#movies data
movies_df=pd.read_csv(r"C:\Users\David\Documents\PHASE 4 PROJECT\movies.csv")
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
#ratings data
ratings_df=pd.read_csv(r"C:\Users\David\Documents\PHASE 4 PROJECT\ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
#links data
links_df=pd.read_csv(r"C:\Users\David\Documents\PHASE 4 PROJECT\links.csv")
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### 2. Data preprocessing 

In [20]:
#Merging DataFrames to create a comprehensive dataset
# Merge ratings with movies to get movie titles
merged_df = pd.merge(ratings_df, movies_df, on='movieId')

# merge links
merged_df = pd.merge(merged_df, links_df, on='movieId')

# Preview the merged DataFrame
print(merged_df.head())


   userId  movieId  rating   timestamp             title  \
0       1        1     4.0   964982703  Toy Story (1995)   
1       5        1     4.0   847434962  Toy Story (1995)   
2       7        1     4.5  1106635946  Toy Story (1995)   
3      15        1     2.5  1510577970  Toy Story (1995)   
4      17        1     4.5  1305696483  Toy Story (1995)   

                                        genres  imdbId  tmdbId  
0  Adventure|Animation|Children|Comedy|Fantasy  114709   862.0  
1  Adventure|Animation|Children|Comedy|Fantasy  114709   862.0  
2  Adventure|Animation|Children|Comedy|Fantasy  114709   862.0  
3  Adventure|Animation|Children|Comedy|Fantasy  114709   862.0  
4  Adventure|Animation|Children|Comedy|Fantasy  114709   862.0  


In [22]:
# Handling missing values.
merged_df.isnull().sum()

userId        0
movieId       0
rating        0
timestamp     0
title         0
genres        0
imdbId        0
tmdbId       13
dtype: int64

There are 13 missing values in tmdbId column. The other  columns have no missing values. I will use forwad fill to handle the missing values.

In [23]:
## forward fill
merged_df['tmdbId'] = merged_df['tmdbId'].fillna(method='ffill')

In [25]:
#checking for data types
merged_df.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
title         object
genres        object
imdbId         int64
tmdbId       float64
dtype: object